In [0]:
%pip install -q google-generativeai

In [0]:
import requests
import google.generativeai as genai
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

In [0]:
# Secrets
GITHUB_TOKEN   = dbutils.secrets.get("mysecrets", "github_token")
JIRA_API_TOKEN = dbutils.secrets.get("mysecrets", "jira_api_token")
GEMINI_API_KEY = dbutils.secrets.get("mysecrets", "gemini_api_key")
JIRA_EMAIL     = dbutils.secrets.get("mysecrets", "jira_email")
JIRA_BASE_URL  = dbutils.secrets.get("mysecrets", "jira_base_url")

# Parameters passed from GitHub Actions 
REPO                = dbutils.widgets.get("repo")
COMMIT_SHA          = dbutils.widgets.get("commit_sha").strip()
BRANCH              = dbutils.widgets.get("branch").strip()
COMMIT_AUTHOR_EMAIL = dbutils.widgets.get("commit_author_email").strip()
COMMIT_MESSAGE      = dbutils.widgets.get("commit_message").strip()
COMMIT_URL          = dbutils.widgets.get("commit_url").strip()

print(f"Ready | {REPO} | {BRANCH} | {COMMIT_SHA or 'latest'} | {COMMIT_AUTHOR_EMAIL} | {COMMIT_URL}")   


In [0]:
import yaml
import os

config_path = "/Workspace/Users/sumit80856@gmail.com/ai_jira_bot/jira_issue_key_config.yaml"

if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config file not found!\nPath: {config_path}\nMake sure the file exists.")

with open(config_path, "r") as f:
    raw_content = f.read()

# Parse only the UNCOMMENTED entries (lines not starting with #)
config = yaml.safe_load(raw_content)

# Get only entries that are not commented out (they will appear in the list)
active_entries = [e for e in config.get("configs", []) if e]  # filters out None from commented blocks

if len(active_entries) == 0:
    raise ValueError("❌ No active Jira ticket found!\nUncomment exactly ONE block in your YAML file.")

if len(active_entries) > 1:
    error_msg = "❌ Multiple Jira tickets active!\nOnly ONE block can be uncommented at a time:\n"
    error_msg += "\n".join(f"  → {e['name']} | {e['issue_key']}" for e in active_entries)
    raise ValueError(error_msg)

JIRA_ISSUE_KEY = active_entries[0]["issue_key"]
JIRA_ISSUE_KEY = active_entries[0]["issue_key"]
print(f"Active Jira Ticket → {JIRA_ISSUE_KEY} — {active_entries[0]['name']}")

In [0]:
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=Retry(total=5, backoff_factor=1)))

def get_commit_diff(owner_repo: str, sha: str) -> str:
    owner, repo = owner_repo.split("/", 1)
    url = f"https://api.github.com/repos/{owner}/{repo}/commits/{sha}"
    headers = {"Authorization": f"token {GITHUB_TOKEN}","Accept": "application/vnd.github.diff"}  # This gives us the full diff directly!
    r = session.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    full_diff = r.text
    
    # Comprehensive ignore list
    config_keywords = [
        ".json", ".env", ".ini", ".conf",
        "config/", "secrets.", "jira_issue_key_config.yaml",
        "requirements.txt", "package.json",
        "README.md", ".gitignore", ".env.example"
    ]
    
    lines = full_diff.splitlines()
    filtered_lines = []
    current_file = None
    
    for line in lines:
        if line.startswith("diff --git"):
            current_file = line.split(" b/")[-1] if " b/" in line else None
            # Skip entire file if it matches config pattern
            if current_file and any(kw in current_file for kw in config_keywords):
                current_file = None  # Mark to skip
                continue
        if current_file:  # Only keep lines from non-config files
            filtered_lines.append(line)
    
    result = "\n".join(filtered_lines) if filtered_lines else ""
    return result[:120000]  # Safety limit for Gemini
    

# Gemini summarizer
def summarize_with_gemini(diff_text: str) -> str:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel("gemini-2.0-flash")
        
        prompt = f"""You are a senior software engineer writing a concise, professional Jira comment.

Summarize the following code changes in exactly 3 bullet points:
- Focus on behavior, logic, or functionality changes
- Ignore filenames, line numbers, whitespace, comments
- Be technical but clear
- Do not use bold, emojis, or headers

Code changes (diff):
{diff_text}

Jira comment (exactly 3 bullets, no bold, no emojis, no headers):"""
        
        response = model.generate_content(prompt)
        return response.text.strip()
    
    except Exception as e:
        return f"GEMINI ERROR: {str(e)}\nManual review required."


# Post to Jira
def post_to_jira(comment: str) -> int:
    url = f"{JIRA_BASE_URL}/rest/api/3/issue/{JIRA_ISSUE_KEY}/comment"
    payload = {
        "body": {
            "type": "doc",
            "version": 1,
            "content": [{"type": "paragraph", "content": [{"type": "text", "text": comment}]}]
        }
    }
    auth = (JIRA_EMAIL, JIRA_API_TOKEN)
    r = session.post(url, json=payload, auth=auth, timeout=30)
    r.raise_for_status()
    return r.status_code

print(f"Fetching code changes for commit {COMMIT_SHA}...")

diff_text = get_commit_diff(REPO, COMMIT_SHA)
print("✓ Diff fetched & config files ignored")
if diff_text=="":
    print("No meaningful code changes (only config/docs/files ignored.)")
else:

    summary = summarize_with_gemini(diff_text)
    print("\nGemini Summary:\n" + summary)

    header = f"Commit {COMMIT_SHA} by {COMMIT_AUTHOR_EMAIL}\n{COMMIT_URL}\n"
    final_comment = header + "\n" + summary

    status = post_to_jira(final_comment)
    print(f"\nPosted to Jira {JIRA_ISSUE_KEY} ✅ Status: {status}")